In [2]:
from __future__ import division

import sys
sys.path.append("../Modules/")
sys.path.append("../")
import os
print(os.getcwd())
import pandas as pd

import numpy as np
from scipy import signal as sig

import performFun as pF
import catchE1Funs as expFun
import recalibration as rc

from plottingFuns import *
from analysisParameters import loadParameters

bkP.output_notebook()

fileTime = '2017-4-14-18-36'

analysisParameters = loadParameters(fileTime)
print(analysisParameters)
startFresh = True
loadProcessed = False

##############################################################################
##############################################################################
### loadSessionDict
# We can just tell members of the audience that any way they can get the data into a dataframe is OK
sessionDict = pF.loadSessionDict(analysisParameters, startFresh=startFresh, loadProcessed=loadProcessed)
print sessionDict.keys()
sessionDict['raw']

SyntaxError: invalid syntax (<ipython-input-2-b2983873bba2>, line 34)

# Calc cyc gaze direction

### Create cyclopean gaze vector

I'm pretending we dont' have cycEyeInHead.  Why?  I don't think the eye tracker gives it to you for free.  Lets begin by dropping it.

In [2]:
sessionDict['raw'] = sessionDict['raw'].drop('cycEyeInHead', axis=1, level=0)

### Take the average of the two vectors, then normalize

In [3]:
# Wow.  This method is actually quite slow compared to the numpy method.  I'm surprised!
cycEyeInHeadDf = sessionDict['raw'].apply(lambda x: (x['rightEyeInHead'] + x['leftEyeInHead']) / 2.0, axis=1)
cycEyeInHeadDf[1:5]

Empty DataFrame
Columns: [(frameNumber, ), (IOD, ), (IPD, ), (ballFinalPos, X), (ballFinalPos, Y), (ballFinalPos, Z), (ballInitialPos, X), (ballInitialPos, Y), (ballInitialPos, Z), (ballInitialVel, X), (ballInitialVel, Y), (ballInitialVel, Z), (ballMat, 0), (ballMat, 1), (ballMat, 2), (ballMat, 3), (ballMat, 4), (ballMat, 5), (ballMat, 6), (ballMat, 7), (ballMat, 8), (ballMat, 9), (ballMat, 10), (ballMat, 11), (ballMat, 12), (ballMat, 13), (ballMat, 14), (ballMat, 15), (ballPos, X), (ballPos, Y), (ballPos, Z), (ballTTC, ), (ballVel, X), (ballVel, Y), (ballVel, Z), (blankDur, ), (blockNumber, ), (calibrationCounter, ), (calibrationPos, X), (calibrationPos, Y), (calibrationPos, Z), (cycEyeBasePoint, X), (cycEyeBasePoint, Y), (cycEyeBasePoint, Z), (cycEyeNodeInWorld, X), (cycEyeNodeInWorld, Y), (cycEyeNodeInWorld, Z), (cycEyeOnScreen, X), (cycEyeOnScreen, Y), (cycGazeNodeInWorld, X), (cycGazeNodeInWorld, Y), (cycGazeNodeInWorld, Z), (cycInverseMat, 0), (cycInverseMat, 1), (cycInverseMat, 2), (cycInverseMat, 3), (cycInverseMat, 4), (cycInverseMat, 5), (cycInverseMat, 6), (cycInverseMat, 7), (cycInverseMat, 8), (cycInverseMat, 9), (cycInverseMat, 10), (cycInverseMat, 11), (cycInverseMat, 12), (cycInverseMat, 13), (cycInverseMat, 14), (cycInverseMat, 15), (cycMat, 0), (cycMat, 1), (cycMat, 2), (cycMat, 3), (cycMat, 4), (cycMat, 5), (cycMat, 6), (cycMat, 7), (cycMat, 8), (cycMat, 9), (cycMat, 10), (cycMat, 11), (cycMat, 12), (cycMat, 13), (cycMat, 14), (cycMat, 15), (eventFlag, ), (eyeTimeStamp, ), (frameTime, ), (inCalibrationQ, ), (isBallVisibleQ, ), (isCalibratedSMIQ, ), (leftEyeBasePoint, X), (leftEyeBasePoint, Y), (leftEyeBasePoint, Z), (leftEyeInHead, X), (leftEyeInHead, Y), (leftEyeInHead, Z), (leftEyeInverseMat, 0), (leftEyeInverseMat, 1), (leftEyeInverseMat, 2), (leftEyeInverseMat, 3), ...]
Index: []

[0 rows x 250 columns]

In [4]:
cycEyeInHeadDf = np.add(sessionDict['raw']['rightEyeInHead'], sessionDict['raw']['leftEyeInHead']) / 2
cycEyeInHeadDf.columns = pd.MultiIndex.from_tuples([('cycEyeInHead','X'),('cycEyeInHead','Y'),('cycEyeInHead','Z')])
cycEyeInHeadDf[1:5]

Empty DataFrame
Columns: [(cycEyeInHead, X), (cycEyeInHead, Y), (cycEyeInHead, Z)]
Index: []

### Normalize the vector

In [ ]:
def normalizeDirection(xyz):
    return xyz/np.linalg.norm(xyz)

cycEyeInHeadDf = cycEyeInHeadDf.apply(lambda xyz: normalizeDirection(xyz), axis=1)

### Add new dataframe to sessionDict['processed']

In [ ]:
sessionDict['processed'] = pd.concat([sessionDict['processed'], cycEyeInHeadDf], axis=1, verify_integrity=1) 

# Now that we have the cyclopean eye-in-head vector, we can apply our head transformation matrix to position it in the real world

## Note that this requires that we know what direction is 'forward' in the SMI glasses space, and in our own head's local coordinate system, wehre straight ahead is 0,0,1.  Up is 0,1,0

Note that this operation requires information spread across two dataframes.
So, as a first step, we must create a temporary dataframe with the columns of data that we will need of our operation

In [ ]:
newDf = pd.concat([sessionDict['processed']['cycEyeInHead'], sessionDict['raw']['viewMat']],
          axis=1, keys=['cycEyeInHead','viewMatrix'])

newDf[1:5]

### Now, lets apply a function to each row that multiplies the data by the transform.  This should be a function.

In [ ]:
# THis is the. only working implementation I've found. Ungh.
for fr in range(0,10): #range(len(sessionDict['raw']['viewMat'])):
    mat = sessionDict['raw']['viewMat'].values[fr].reshape([4,4]).T
    xyzw = np.append(sessionDict['processed']['cycEyeInHead'].values[fr], True) # The True converst into a 1:  [x y z 1]
    print np.dot(mat,xyzw)[0:3]


In [ ]:
out_fr_XYZ = [np.dot(sessionDict['raw']['viewMat'].values[fr].reshape([4,4]).T,
                     np.append(sessionDict['processed']['cycEyeInHead'].values[fr],1)) for fr in range(0,10)]

out_fr_XYZ

## I really want to solve this in a way that will allow use of the apply command.

### Here are some failed attempts

In [ ]:
# Attempt one
#newDf.apply(lambda row: np.dot(row['viewMatrix'].values.reshape([4,4]).T,[0,0,1.0,0]),axis=1)



# Attempt two
#def multByMatrix(matrix_fr,xyz_fr,translate = True):       
#     out_fr_XYZ = np.array([ np.dot(matrix_fr.values[fr].reshape([4,4]).T,
#                                    np.append(xyz_fr.values[fr],translate)) 
#                            for fr in range(len(xyz_fr))])



### Eureeka!  It turns out that the output from the function must be a tuple, or you get the error....
ValueError: Shape of passed values is (40415, 3), indices imply (40415, 19)


In [ ]:
# Fuck it!  We'll do it in a for loop.
def multByMatrix(matrix, xyz, translate=True):
        
    mat = matrix.values.reshape([4,4]).T
    xyzw = np.append(xyz,True)
    xyzOut = np.dot(mat, xyzw)
    
    # It seems to be an issue of the return type
    #return xyzOut[0:3] # THis does not work
    #return [1,2,3] # THis does

    #Eureeka!  You must return a tuple. This is pretty fast.
    return (xyzOut[0], xyzOut[1], xyzOut[2])



tranformedXYZ = newDf.apply(lambda row: multByMatrix(row['viewMatrix'],row['cycEyeInHead']),axis=1)
tranformedXYZ[1:10]

# Do this for a few more things.  Plot. Success!